# Сбор связей

In [ ]:
from collections import Counter, defaultdict
from nlp import nlp
from dawg import CompletionDAWG


def filter_deps(word):
    return word.deprel in ["amod"] # , "appos"


def find_deps(words):
    deps = []
    
    for w in words:
        if w.upos == "NOUN":
            deps.append((w, [rw for rw in words if rw.head == w.id and filter_deps(rw)]))
    
    return deps


def format_dep(word):
    return f"{word.lemma}:{word.deprel}:{word.upos}"


def add_deps(all_deps, deps):
    for (root, dep_words) in deps:
        if dep_words:
            all_deps[root.lemma] += Counter(format_dep(dw) for dw in dep_words)


if not 'doc' in locals():
    response = requests.get('https://raw.githubusercontent.com/alexander-karpov/alice-skill-repka/master/data/books/Bianki_Vitalij_-_Glaza_i_ushi.txt')
    text = response.text
    doc = nlp(text)


deps = defaultdict(Counter)

for sent in doc.sentences:
    add_deps(deps, find_deps(sent.words))
    


completion_dawg = CompletionDAWG([f"{root}:{dep}" for root in deps for dep in deps[root]])


In [58]:
completion_dawg.keys("шея")

['шея:белый:amod:ADJ', 'шея:гордый:amod:ADJ']

# Распознание сущностей

In [ ]:
from nlp import nlp
from ipymarkup import show_dep_markup, show_span_line_markup
import requests
    

response = requests.get('https://raw.githubusercontent.com/alexander-karpov/alice-skill-repka/master/data/books/Bianki_Vitalij_-_Glaza_i_ushi.txt')
text = response.text

if not 'doc' in locals():
    doc = nlp(text)
    
    
def filter_deps(word):
    if word.upos in [
        # Пунктуация
        "PUNCT",
        # Приставки
        "ADP",
        # Детерминант (непонятная хрень)
        "DET",
        # Частицы
        "PART",
        # Наречия
        "ADV",
        # Цифры
        "NUM",
    ]:
        return False
    
    if word.deprel in [
        # Входы и<-cc conj->выходы
        "conj",
        # Союз
        "cc",
        # Непонятная связь
        "parataxis",
        # Непонятная связь
        "acl",
        # Как, будто
        "mark",
    ]:
        return False
    
    
    """
    Взяль nmod часто определяется неправильно/не так как нам нужно
    
    Должны быть варианты типа:
    карта стурента
    каска шахтёра
    луг за nmod:NOUNдеревней
    лодочка на nmod:NOUNножках
    
    Но попадаются варианты:
    солнышко на nmod:NOUN закат
    листок на nmod:NOUN луг
    муравьишка себе nmod:NOUN ноги
    коленки у nmod:NOUN сенокосца
    брюхом по nmod:NOUN земле
    ботва для nmod:NOUN муравьишки
    спиной nmod:NOUN крылья
    рылья у nmod:NOUN жука
    Муравьишка с nmod:NOUN листовёрткой
    Муравьишка с nmod:NOUN гусеницы
    кувырк с nmod:NOUN листа
    гусеница nmod:NOUN лапки
    """
    if word.deprel == 'nmod':
        return False
    
    
    return True

def find_entity(words):
    nouns = []
    
    for w in words:
        if w.upos == "NOUN":
            nouns.append((
                w,
                [rw for rw in words if rw.head == w.id and filter_deps(rw)]
            ))
    
    found = False
    
        
    words = ['-', *(w.text for w in words)]
    deps = []
    
    for (r, ds) in nouns:
#         deps.append((0, r.id, r.deprel))
        
        for d in ds:
            found = True
            deps.append((d.head, d.id, f"{d.deprel}:{d.upos}"))

#     if found:
    show_dep_markup(words, deps)
    

for sent in doc.sentences:
    find_entity(sent.words)

"""
гусеница - землемер
паук-сенокосец
шестиногий конь

"""

# Сохранилка

```
def filter_deps(word):
    if word.upos in [
        # Пунктуация
        "PUNCT",
        # Приставки
        "ADP",
        # Детерминант (непонятная хрень)
        "DET",
        # Частицы
        "PART",
        # Наречия
        "ADV",
        # Цифры
        "NUM",
    ]:
        return False
    
    if word.deprel in [
        # Входы и<-cc conj->выходы
        "conj",
        # Союз
        "cc",
        # Непонятная связь
        "parataxis",
        # Непонятная связь
        "acl",
        # Как, будто
        "mark",
    ]:
        return False
    
    
    """
    Взяль nmod часто определяется неправильно/не так как нам нужно
    
    Должны быть варианты типа:
    карта стурента
    каска шахтёра
    луг за nmod:NOUNдеревней
    лодочка на nmod:NOUNножках
    
    Но попадаются варианты:
    солнышко на nmod:NOUN закат
    листок на nmod:NOUN луг
    муравьишка себе nmod:NOUN ноги
    коленки у nmod:NOUN сенокосца
    брюхом по nmod:NOUN земле
    ботва для nmod:NOUN муравьишки
    спиной nmod:NOUN крылья
    рылья у nmod:NOUN жука
    Муравьишка с nmod:NOUN листовёрткой
    Муравьишка с nmod:NOUN гусеницы
    кувырк с nmod:NOUN листа
    гусеница nmod:NOUN лапки
    """
    if word.deprel == 'nmod':
        return False
```
